In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['Google AI Key'] = os.getenv('GOOGLE_API_KEY')

In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("LangChain_PDF_QA_Chatbot_Guide.pdf")

document=loader.load()

In [3]:
print(document)

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-01-02T14:23:42+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-01-02T14:23:42+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'LangChain_PDF_QA_Chatbot_Guide.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Building a PDF-Based Q&A; Chatbot using LangChain\n1. Introduction\nThis document is designed as an ideal knowledge source for building a Question & Answer chatbot\nusing LangChain. It is structured, text-based, and optimized for document loading, chunking, and\nvector retrieval.\n2. What is a Q&A; Chatbot?\nA Q&A; chatbot is an AI system that answers user questions by retrieving relevant information from\na knowledge base and generating accurate responses using a Large Language Model (LLM).\n3. LangChain Overview\nLangChain is a framework that helps developers build application

In [4]:
#import langchain memory
from langchain_classic.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

C:\Users\PC\AppData\Local\Temp\ipykernel_23112\1944106899.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,    # Maximum size of each chunk
                                             chunk_overlap=0)    # Number of characters to overlap between chunks
docs=text_splitter.split_documents(document)

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

# Free local embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [7]:
from langchain_community.vectorstores import FAISS  
vectorstore = FAISS.from_documents(docs, embeddings)            # Create a FAISS vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})    # Create a retriever and k means How many similar chunks to return

In [8]:
system_prompt = """
You are a helpful assistant. 
Answer questions based on the documents provided by the retriever. 
If the information is not in the documents, answer from your own knowledge, 
but clearly indicate when you are doing so.
"""

In [9]:
from langchain_google_genai import GoogleGenerativeAI


llm=GoogleGenerativeAI(temperature=0, model="models/gemini-2.5-flash-lite", 
                       )

In [10]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(  # Create a RetrievalQA chain and from_chain_type means use the specified chain type
    llm=llm,
    retriever=retriever,  
    chain_type="stuff" ,   # Use the "stuff" chain type to combine information from multiple sources
    memory=memory,

)


In [11]:
query = "What is langchain?"
response = qa_chain.invoke(query)
print(response['result'])

LangChain is a framework that helps developers build applications powered by language models. It supports document loading, text splitting, embeddings, vector databases, and retrieval-augmented generation (RAG).
